In [1]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [2]:
(train_image, train_labels), (test_image, test_labels) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 20s 2us/step


In [3]:
train_count = train_image.shape[0]
train_image.shape
# 扩张维度
train_image = tf.expand_dims(train_image, -1)
# 改变数据类型
train_image = tf.cast(train_image/255, tf.float32)
# 测试数据扩张维度
test_image = tf.expand_dims(test_image, -1)
# 改变数据类型
test_image = tf.cast(test_image/255, tf.float32)
train_labels = tf.cast(train_labels, tf.int64)
train_labels
test_labels = tf.cast(test_labels, tf.int64)
test_labels
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_labels))

In [4]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_labels))

In [5]:
dataset = dataset.shuffle(10000).batch(32)
test_dataset = test_dataset.batch(32)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3],activation="relu",input_shape = (28,28,1)),
    tf.keras.layers.Conv2D(32,[3,3],activation="relu"),
    tf.keras.layers.Conv2D(64,[3,3],activation="relu"),
    tf.keras.layers.Conv2D(128,[3,3],activation="relu"),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10,activation='softmax',name="output_class")
])

In [7]:
# 模型的训练参数
model.trainable_variables

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 16) dtype=float32, numpy=
 array([[[[ 0.04314114, -0.13022056,  0.09248827,  0.14643441,
           -0.1671805 ,  0.1929857 ,  0.08097501,  0.06246565,
            0.02783878, -0.0225317 ,  0.19552253, -0.05367036,
            0.09222944, -0.14710808,  0.10481839,  0.10019533]],
 
         [[ 0.12636788,  0.00967041,  0.05626325, -0.01264162,
            0.15506957, -0.01200745,  0.15493058, -0.1066279 ,
            0.1609105 ,  0.13111602,  0.0980957 , -0.09080642,
            0.16896783,  0.14135651,  0.15078972,  0.12223218]],
 
         [[-0.0674717 , -0.10517069,  0.16739576, -0.16523108,
            0.08976178,  0.02398774, -0.00455005,  0.14992158,
           -0.17983463,  0.02787392,  0.14370357, -0.04674272,
           -0.17902812, -0.18235354,  0.01248549, -0.1958815 ]]],
 
 
        [[[-0.02253279,  0.17010336, -0.10661586, -0.02281207,
            0.12322439, -0.05319332, -0.17862236, -0.13363066,
           -0.06024717, -0.07

In [8]:
optimizer = tf.keras.optimizers.Adam()
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [9]:
features, labels = next(iter(dataset))

In [10]:
predictions = model(features)

In [11]:
tf.argmax(predictions, axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([8, 8, 8, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 2, 2, 2, 8, 8, 8], dtype=int64)>

In [12]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([9, 3, 7, 9, 3, 2, 9, 1, 0, 4, 4, 7, 7, 2, 0, 6, 0, 0, 0, 7, 9, 7,
       4, 8, 3, 3, 6, 0, 0, 3, 2, 9], dtype=int64)>

In [13]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy') 
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels, pred)
def test_step(model, images, labels):
    pred = model(images)
    loss_step = loss_func(labels, pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)
def train():
    for epoch in range(100):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        print('Epoch{} loss is {}, accuracy is {}'.format(epoch,
                                                          train_loss.result(),
                                                          train_accuracy.result()))
        
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(model, images, labels)
        print('Epoch {} test_loss is {}, test_accuracy is {}'.format(epoch,
                                                              test_loss.result(),
                                                              test_accuracy.result()))
        
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [14]:
train() # 这时候就开始运算了。就是有点慢QAQ

Epoch0 loss is 1.818490982055664, accuracy is 0.6482166647911072
Epoch 0 test_loss is 1.6532930135726929, test_accuracy is 0.8123999834060669
Epoch1 loss is 1.6348069906234741, accuracy is 0.8288666605949402
Epoch 1 test_loss is 1.6289761066436768, test_accuracy is 0.8323000073432922
Epoch2 loss is 1.609864354133606, accuracy is 0.8520833253860474
Epoch 2 test_loss is 1.5935641527175903, test_accuracy is 0.8673999905586243
Epoch3 loss is 1.5969042778015137, accuracy is 0.8646833300590515
Epoch 3 test_loss is 1.633488416671753, test_accuracy is 0.8278999924659729
Epoch4 loss is 1.5896588563919067, accuracy is 0.8715999722480774
Epoch 4 test_loss is 1.5783663988113403, test_accuracy is 0.8830000162124634
Epoch5 loss is 1.584984302520752, accuracy is 0.8761000037193298
Epoch 5 test_loss is 1.5738787651062012, test_accuracy is 0.887499988079071
Epoch6 loss is 1.5137780904769897, accuracy is 0.9484333395957947
Epoch 6 test_loss is 1.4881161451339722, test_accuracy is 0.9732999801635742
Epoc

Epoch 57 test_loss is 1.473874568939209, test_accuracy is 0.9872999787330627
Epoch58 loss is 1.4711859226226807, accuracy is 0.989883303642273
Epoch 58 test_loss is 1.4715040922164917, test_accuracy is 0.9900000095367432
Epoch59 loss is 1.4716421365737915, accuracy is 0.989466667175293
Epoch 59 test_loss is 1.4743565320968628, test_accuracy is 0.9864000082015991
Epoch60 loss is 1.4717605113983154, accuracy is 0.9893333315849304
Epoch 60 test_loss is 1.484956979751587, test_accuracy is 0.9760000109672546
Epoch61 loss is 1.4703800678253174, accuracy is 0.9907000064849854
Epoch 61 test_loss is 1.4757341146469116, test_accuracy is 0.9854000210762024
Epoch62 loss is 1.471519947052002, accuracy is 0.9896166920661926
Epoch 62 test_loss is 1.4749889373779297, test_accuracy is 0.9860000014305115
Epoch63 loss is 1.471441626548767, accuracy is 0.9896666407585144
Epoch 63 test_loss is 1.4827401638031006, test_accuracy is 0.9782999753952026
Epoch64 loss is 1.4706186056137085, accuracy is 0.99053335

In [1]:
# 保存模型
model.save("./model/mmmmmm.h5")

NameError: name 'model' is not defined